In several quantum machine learning algorithms, an important sub-routine is amplitude encoding. This refers to the process whereby classical data is encoded into the amplitudes of a quantum register. A classical data point must consist of numerical values, which should then be normalized so that the sum of the square of the components sum to 1. Thereafter, the (sub-)routine described below can be used to encode that classical datapoint into a quantum state.

In [1]:
import numpy as np
import math

from pyquil import Program, get_qc
from pyquil.api import WavefunctionSimulator
from pyquil.gates import *

import itertools

Given a quantum state with real amplitudes, how do we construct a circuit that would prepare such a state? As described in "Transformation of quantum states using uniformly controlled rotations" by Mottonen et al. (https://arxiv.org/pdf/quant-ph/0407010.pdf), we need to run the following series of controlled rotations in reverse:

![Drawing](https://raw.githubusercontent.com/QuNovaComputing/Hackathon2021/qunovacomputing/Qunova%20Computing/rigetti_resources/uniformly_controlled_rotations.png)

where the black dots denote the control qubit being in the state $\vert 1 \rangle$ and the white dots denote the control qubit being in the state $\vert 0 \rangle$.

The angles are to be calculated according to

![Drawing](https://raw.githubusercontent.com/QuNovaComputing/Hackathon2021/qunovacomputing/Qunova%20Computing/rigetti_resources/beta_angles.png)

(images borrowed from "Supervised Learning with Quantum Computers" by M. Schuld, F. Petruccione)

Let us first write a function that can calculate the $\beta$'s above.

In [2]:
def all_betas(amps):
    """
    Given some real amplitudes, compute the RY angles needed to prepare this state

    :return dict: key: (s, j), value: beta angle
    """
    n = math.log(len(amps), 2)
    assert np.isclose(n, int(n)), "Specify 2^n amplitudes for some n"
    n = int(n)
    d_betas = {}
    for s in range(1, n + 1):
        for j in range(1, 2**(n-s) + 1):
            # calculate numerator
            numer_sqr = 0.0
            for l in range(2**(s - 1)):
                idx_num = (2 * j - 1) * (2**(s-1)) + l
                numer_sqr += np.abs(amps[idx_num])**2
            numerator = np.sqrt(numer_sqr)
            # calculate denominator
            denom_sqr = 0.0
            for ll in range(2**s):
                idx_den = (j - 1) * (2**s) + ll
                denom_sqr += np.abs(amps[idx_den])**2
            denominator = np.sqrt(denom_sqr)
            # avoid any pathological cases, e.g. if denominator = 0.0
            if np.isclose(numerator, 0.0):
                ratio = 0.0
            else:
                ratio = numerator / denominator
            # ensure argument to arccos lies within domain [-1, 1]
            if ratio > 1.0:
                ratio = 1.0
            elif ratio < -1.0:
                ratio = -1.0
            else:
                pass
            # finally, compute the beta angles
            d_betas[s, j] = -2 * np.arcsin(ratio)
    return d_betas

Following this, we now write a function that will encode a given classical vector into the amplitudes of a quantum state.

In [3]:
def state_prep_prog(amps):
    """
    Given some real amplitudes, compute the pyQuil Program needed to prepare this state
    
    :return Program: state preparation circuit
    """
    n = math.log(len(amps), 2)
    assert np.isclose(n, int(n)), "Specify 2^n amplitudes for some n"
    n = int(n)
    d_betas = all_betas(amps)
    tot_prog = Program()
    for s in range(n, 0, -1):
        tot_js = 2**(n-s)
        num_combs = math.log(tot_js, 2)
        assert np.isclose(num_combs, int(num_combs)), "Something went wrong"
        num_combs = int(num_combs)
        all_combs = np.array(list(itertools.product([0, 1], repeat=num_combs)))
        for j in range(1, tot_js + 1)[::-1]:
            if len(all_combs) == 1:
                tot_prog += Program(f"RY({d_betas[s, j]}) {s-1}")
            else:
                # pick the relevant combination, e.g. [0,0] or [0, 1] or [1, 0] or [1, 1] for two control qubits
                comb = all_combs[j-1]
                rot_prog_str = f"RY({d_betas[s, j]}) {s-1}"
                rot_oper_prog_str = f"RY({d_betas[s, j]}) "
                rot_qub_prog_str = f"{s-1}"
                flip_prog_strs = []
                # this loops through the controlled operation, e.g. [0, 1] in the opposite direction
                for x, cbit in enumerate(comb[::-1]):
                    if cbit == 0:
                        flip_prog_strs += [f"X {s+x}"]
                    else:
                        pass
                    rot_oper_prog_str = "CONTROLLED " + rot_oper_prog_str
                    rot_qub_prog_str = f"{s+x} " + rot_qub_prog_str
                rot_prog_str = rot_oper_prog_str + rot_qub_prog_str
                tot_prog += Program(flip_prog_strs) + Program(rot_prog_str) + Program(flip_prog_strs[::-1])

    return tot_prog

To check the correctness of our implementation, we will test the functions above against random amplitudes.

In [4]:
wfn_sim = WavefunctionSimulator()

In [5]:
def random_amplitudes(num_qubits=3):
    N = 2**num_qubits
    amps = np.random.uniform(low=0.0, high=1.0, size=N)
    amps *= 1 / np.sqrt(np.sum(np.abs(amps)**2))
    assert np.isclose(np.sum(np.abs(amps)**2), 1.0), "Amplitudes do not square-sum to 1"
    return amps

In [6]:
# draw some random real amplitudes
amps = random_amplitudes()

# create the state prep Program corresponding to these amplitudes
prep_prog = state_prep_prog(amps)
wfn = wfn_sim.wavefunction(prep_prog)

# compare the outcome probabilities resulting from the state prep Program
# with the expected probabilities
outcome_probs = list(wfn.get_outcome_probs().values())
expected_probs = amps**2
assert np.allclose(expected_probs, outcome_probs), "Measurement probabilities do not agree"